<a href="https://colab.research.google.com/github/digwit678/DIGCREAT_AUDIO_PROCESSION/blob/main/baseline_xlm_roberta_large_xnli_colab_train_trials_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[based on this notebook](https://www.kaggle.com/code/yehoonjoo/nli-with-xlm-roberta#NLI-with-XLM-RoBERTa)

***main differences***
 
- HSLU data used

- slightly different model used

# Imports

In [1]:
!pip install transformers
!pip install sacremoses
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import torch as torch

from pandas import read_csv
import numpy as np
# set a seed value
torch.manual_seed(555)

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = "cuda"
train = read_csv("train.csv")
evaluation = read_csv("valid.csv")
test = read_csv("test.csv")

# Preprocessing

## Drop and Rename Features

In [3]:
train.head()

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1


***Observation(s)***

- The id column seems unnecessary as it does not include useful information.
- The lang_abv and language columns seem unnecessary as the XLM-RoBERTa transformer is a multilingual model which "does not require lang tensors to understand which language is used."

***Decision(s)***

- Drop the id, lang_abv, and language columns from the dataset

from [this notebook](https://www.kaggle.com/code/yehoonjoo/nli-with-xlm-roberta#NLI-with-XLM-RoBERTa)

In [4]:
train = train.drop(["id","lang_abv","language"], axis='columns')

# Tokenizing with xlm-mlm-xnli15-1024

In [5]:
import torch
from transformers import XLMTokenizer
from transformers import AutoTokenizer
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification

MODEL_TYPE = 'joeddav/xlm-roberta-large-xnli'

# xlm-roberta-large
print('Loading XLMRoberta tokenizer...')

tokenizer_nli = AutoTokenizer.from_pretrained(MODEL_TYPE)

#tokenizer_nli = AutoTokenizer.from_pretrained(MODEL_TYPE)

Loading XLMRoberta tokenizer...


In [6]:
tokenizer_nli

PreTrainedTokenizerFast(name_or_path='joeddav/xlm-roberta-large-xnli', vocab_size=250002, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'})

In [7]:
# TODO: maybe change this value
max_input_length = 256

## Finetune Format

## batch_sentences example

In [8]:
batch_sentences = [
    "But what about second breakfast?",
    "Don't think he knows about second breakfast, Pip.",
    "What about elevensies?",
]
encoded_inputs = tokenizer_nli(batch_sentences, padding=True, truncation=True)
print(encoded_inputs)

# to which length were the sentences padded ?
encoded_lengths = encoded_inputs.input_ids
print("\n padding length: \n")
print([len(sentence) for sentence in encoded_lengths])

{'input_ids': [[0, 4966, 2367, 1672, 17932, 118285, 32, 2, 1, 1, 1, 1, 1, 1], [0, 7650, 25, 18, 5351, 764, 93002, 1672, 17932, 118285, 4, 93128, 5, 2], [0, 4865, 1672, 103155, 17089, 32, 2, 1, 1, 1, 1, 1, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]]}

 padding length: 

[14, 14, 14]


In [9]:
decoded_inputs = [tokenizer_nli.decode(encoded_inputs["input_ids"][idx]) for idx, input in enumerate(encoded_inputs) ]
print(decoded_inputs)


['<s> But what about second breakfast?</s><pad><pad><pad><pad><pad><pad>', "<s> Don't think he knows about second breakfast, Pip.</s>"]


## Preprocess Dataframe

In [10]:
train_copy = train.copy()
train


,premise,hypothesis,label
0,and these comments were considered in formulat...,The rules developed in the interim were put to...,0
1,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,2
2,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,0
3,you know they can't really defend themselves l...,They can't defend themselves because of their ...,0
4,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,1
...,...,...,...
10903,The results of even the most well designed epi...,All studies have the same amount of uncertaint...,2
10904,But there are two kinds of the pleasure of do...,But there are two kinds of the pleasure of doi...,0
10905,The important thing is to realize that it's wa...,"It cannot be moved, now or ever.",2
10906,At the west end is a detailed model of the who...,The model temple complex is at the east end.,2


[SOURCE: basics-of-bert-and-xlm-roberta-pytorch](https://www.kaggle.com/code/naim99/basics-of-bert-and-xlm-roberta-pytorch)

# Encoder Class

In [11]:
from torch.utils.data import Dataset, DataLoader
MAX_LEN = max_input_length
class CompDataset(Dataset):

    def __init__(self, df):
        self.df_data = df



    def __getitem__(self, index):

        # get the sentence from the dataframe
        sentence1 = self.df_data.loc[index, 'premise']
        sentence2 = self.df_data.loc[index, 'hypothesis']

        # Process the sentence
        # ---------------------

        encoded_dict = tokenizer_nli.encode_plus(
            sentence1, sentence2,           # Sentences to encode.
            add_special_tokens = True,      # Add the special tokens.
            max_length = MAX_LEN,           # Pad & truncate all sentences.
            pad_to_max_length = True,
            return_attention_mask = True,   # Construct attn. masks.
            return_tensors = 'pt',          # Return pytorch tensors.
        )
        #print(encoded_dict)

        # These are torch tensors.
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]

        # Convert the target to a torch tensor
        target = torch.tensor(self.df_data.loc[index, 'label'])

        sample = (padded_token_list, att_mask, target)


        return sample


    def __len__(self):
        return len(self.df_data)






class TestDataset(Dataset):

    def __init__(self, df):
        self.df_data = df



    def __getitem__(self, index):

        # get the sentence from the dataframe
        sentence1 = self.df_data.loc[index, 'premise']
        sentence2 = self.df_data.loc[index, 'hypothesis']

        # Process the sentence
        # ---------------------

        encoded_dict = tokenizer_nli.encode_plus(
            sentence1, sentence2,           # Sentence to encode.
            add_special_tokens = True,      # Add the special tokens.
            max_length = MAX_LEN,           # Pad & truncate all sentences.
            pad_to_max_length = True,
            return_attention_mask = True,   # Construct attn. masks.
            return_tensors = 'pt',          # Return pytorch tensors.
        )

        # These are torch tensors.
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]



        sample = (padded_token_list, att_mask)


        return sample


    def __len__(self):
        return len(self.df_data)

# KFold

In [12]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.utils import shuffle
# shuffle
df = shuffle(train)

# initialize kfold
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1024)

# for stratification
y = df['label']

# Note:
# Each fold is a tuple ([train_index_values], [val_index_values])
# fold_0, fold_1, fold_2, fold_3, fold_5 = kf.split(df, y)

# Put the folds into a list. This is a list of tuples.
fold_list = list(kf.split(df, y))

train_df_list = []
val_df_list = []

for i, fold in enumerate(fold_list):

    # map the train and val index values to dataframe rows
    df_train = df[df.index.isin(fold[0])]
    df_val = df[df.index.isin(fold[1])]
    
    train_df_list.append(df_train)
    val_df_list.append(df_val)
    
    

print(len(train_df_list))
print(len(val_df_list))

5
5


# Training

In [13]:
import os



NUM_EPOCHS = 3
BATCH_SIZE = 4
NUM_CORES = os.cpu_count()

train_data = CompDataset(train)
val_data = CompDataset(evaluation)
test_data = TestDataset(test)

train_dataloader = torch.utils.data.DataLoader(train_data,
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               num_workers=NUM_CORES)

val_dataloader = torch.utils.data.DataLoader(val_data,
                                             batch_size=BATCH_SIZE,
                                             shuffle=True,
                                             num_workers=NUM_CORES)

test_dataloader = torch.utils.data.DataLoader(test_data,
                                              batch_size=BATCH_SIZE,
                                              shuffle=False,
                                              num_workers=NUM_CORES)

In [14]:
print(len(train_dataloader))
print(len(val_dataloader))
print(len(test_dataloader))

2727
303
1299


In [15]:
print(train_data.__getitem__(1))
print(len(train_data.__getitem__(1)))
# only for GPU training
#print(train_data.__getitem__(1)[0].is_cuda)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


(tensor([     0,  32255,    621,  37348,    450,    642,    148,  56644,    133,
           678,     23,  41361,  94407,    111,  27165,  11037,      7,      4,
          2412,   2804,      5,      6,      2,      2, 109613,     13,  94407,
           621,    959,  28897,   3674,     47,   4488,     98,   6097,  37348,
             5,      2,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [16]:

print(NUM_CORES)

4


In [17]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [19]:
stop

NameError: ignored

In [25]:

model = AutoModelForSequenceClassification.from_pretrained('joeddav/xlm-roberta-large-xnli',      
    num_labels = 3, # The number of output labels. 2 for binary classification.
)
# Send the model to the device.


Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
model.to(device)

NameError: ignored

In [ ]:
# Create a batch of train samples
# We will set a small batch size of 8 so that the model's output can be easily displayed.

train_dataloader = torch.utils.data.DataLoader(train_data,
                                              batch_size=8,
                                              shuffle=True,
                                              num_workers=NUM_CORES)
#
b_input_ids, b_input_mask, b_labels = next(iter(train_dataloader))

print(b_input_ids.shape)
print(b_input_mask.shape)
print(b_labels.shape)

model prediction on one batch of train samples 

In [ ]:
# Pass a batch of train samples to the model.

batch = next(iter(train_dataloader))
# Send the data to the device
b_input_ids = batch[0].to(device)
b_input_mask = batch[1].to(device)
b_labels = batch[2].to(device)

# Run the model
outputs = model(b_input_ids,
                attention_mask=b_input_mask,
                labels=b_labels)

# The ouput is a tuple (loss, preds).
outputs

In [ ]:
# The output is a tuple: (loss, preds)

len(outputs)

In [ ]:
# This is the loss.

outputs[0]

In [ ]:
# These are the predictions.

outputs[1]

In [ ]:
preds = outputs[1].detach().cpu().numpy()

y_true = b_labels.detach().cpu().numpy()
y_pred = np.argmax(preds, axis=1)

y_pred

In [23]:
# This is the accuracy without fine tuning.

from sklearn.metrics import roc_auc_score, accuracy_score
val_acc = accuracy_score(y_true, y_pred)

val_acc

NameError: ignored

In [ ]:
# The loss and preds are Torch tensors

print(type(outputs[0]))
print(type(outputs[1]))

In [22]:
# Define the optimizer

from transformers import AdamW
#from torch import AdamW

# Saving 5 TPU models will exceed the 4.9GB disk space.
# Therefore, will will only train on 3 folds.


L_RATE = 1e-5

optimizer = AdamW(model.parameters(),
              lr = L_RATE, 
              eps = 1e-8 
            )

NameError: ignored

In [21]:
NUM_FOLDS = 5

# Saving 5 TPU models will exceed the 4.9GB disk space.
# Therefore, will will only train on 3 folds.
NUM_FOLDS_TO_TRAIN = 3 

In [26]:
from koila import LazyTensor, lazy

In [27]:
%%time

import random 
# Set a seed value.
seed_val = 1024

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)



# Store the accuracy scores for each fold model in this list.
# [[model_0 scores], [model_1 scores], [model_2 scores], [model_3 scores], [model_4 scores]]
# [[ecpoch 1, epoch 2, ...], [ecpoch 1, epoch 2, ...], [ecpoch 1, epoch 2, ...], [ecpoch 1, epoch 2, ...], [ecpoch 1, epoch 2, ...]]

# Create a list of lists to store the val acc results.
# The number of items in this list will correspond to
# the number of folds that the model is being trained on.
fold_val_acc_list = []
for i in range(0, NUM_FOLDS):
    
    # append an empty list
    fold_val_acc_list.append([])
    
    
    
    

# For each epoch...
for epoch in range(0, NUM_EPOCHS):
    
    print("\nNum folds used for training:", NUM_FOLDS_TO_TRAIN)
    print('======== Epoch {:} / {:} ========'.format(epoch + 1, NUM_EPOCHS))
    
    # Get the number of folds (5 for this example)
    num_folds = len(train_df_list)
    
    # For this epoch, store the val acc scores for each fold in this list.
    # We will use this list to calculate the cv at the end of the epoch.
    epoch_acc_scores_list = []
    
    # For each fold...
    for fold_index in range(0, NUM_FOLDS_TO_TRAIN):
        
        print('\n== Fold Model', fold_index)
        
        
        # .........................
        # Load the fold model
        # .........................
        
        if epoch == 0:
            
            # define the model
            model = AutoModelForSequenceClassification.from_pretrained('joeddav/xlm-roberta-large-xnli',
            num_labels = 3,       
            output_attentions = False, 
            output_hidden_states = False,
            )
            
            # Send the model to the device.
            model.to(device)
            
            optimizer = AdamW(model.parameters(),
              lr = L_RATE, 
              eps = 1e-8
            )
            
        else:
        
            # Get the fold model
            path_model = 'model_' + str(fold_index) + '.bin'
            model.load_state_dict(torch.load(path_model))

            # Send the model to the device.
            model.to(device)
        
        
        
        # .....................................
        # Set up the train and val dataloaders
        # .....................................
        

        # Intialize the fold dataframes
        df_train = train_df_list[fold_index]
        df_val = val_df_list[fold_index]

        
            
        # Reset the indices or the dataloader won't work.
        df_train = df_train.reset_index(drop=True)
        df_val = df_val.reset_index(drop=True)
    
        # Create the dataloaders
        train_data = CompDataset(df_train)
        val_data = CompDataset(df_val)

        train_dataloader = torch.utils.data.DataLoader(train_data,
                                                batch_size=BATCH_SIZE,
                                                shuffle=True,
                                               num_workers=NUM_CORES)

        val_dataloader = torch.utils.data.DataLoader(val_data,
                                                batch_size=BATCH_SIZE,
                                                shuffle=True,
                                               num_workers=NUM_CORES)
    
    


       

        # ========================================
        #               Training
        # ========================================
        
        stacked_val_labels = []
        targets_list = []

        print('Training...')

        # put the model into train mode
        model.train()

        # This turns gradient calculations on and off.
        torch.set_grad_enabled(True)


        # Reset the total loss for this epoch.
        total_train_loss = 0

        for i, batch in enumerate(train_dataloader):

            train_status = 'Batch ' + str(i+1) + ' of ' + str(len(train_dataloader))

            print(train_status, end='\r')
            (lazy_input, lazy_label) = lazy((batch[0],batch[1]), batch[2], batch=0)
            
            b_input_ids = lazy_input[0].to(device)
            b_input_mask = lazy_input[1].to(device)
            #b_token_type_ids = batch[2].to(device)
            b_labels = lazy_label.to(device)
            model.zero_grad()        


            outputs = model(b_input_ids,
                        attention_mask=b_input_mask,
                        labels=b_labels)
            

            # Get the loss from the outputs tuple: (loss, logits)
            loss = outputs[0]
            # Convert the loss from a torch tensor to a number.
            # Calculate the total loss.
            total_train_loss = total_train_loss + loss.item()

            # Zero the gradients
            optimizer.zero_grad()

            # Perform a backward pass to calculate the gradients.
            loss.backward()
            
            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Use the optimizer to update Weights
            
            # Optimizer for GPU
            optimizer.step() 
            
            # Optimizer for TPU
            # https://pytorch.org/xla/
            #xm.optimizer_step(optimizer, barrier=True)
            
           


        print('Train loss:' ,total_train_loss)


        # ========================================
        #               Validation
        # ========================================

        print('\nValidation...')

        # Put the model in evaluation mode.
        model.eval()

        # Turn off the gradient calculations.
        # This tells the model not to compute or store gradients.
        # This step saves memory and speeds up validation.
        torch.set_grad_enabled(False)


        # Reset the total loss for this epoch.
        total_val_loss = 0


        for j, val_batch in enumerate(val_dataloader):

            val_status = 'Batch ' + str(j+1) + ' of ' + str(len(val_dataloader))

            print(val_status, end='\r')

            b_input_ids = val_batch[0].to(device)
            b_input_mask = val_batch[1].to(device)
            #b_token_type_ids = val_batch[2].to(device)
            b_labels = val_batch[2].to(device)      


            outputs = model(b_input_ids,
                    attention_mask=b_input_mask, 
                    labels=b_labels)

            # Get the loss from the outputs tuple: (loss, logits)
            loss = outputs[0]

            # Convert the loss from a torch tensor to a number.
            # Calculate the total loss.
            total_val_loss = total_val_loss + loss.item()

            # Get the preds
            preds = outputs[1]


            # Move preds to the CPU
            val_preds = preds.detach().cpu().numpy()

            # Move the labels to the cpu
            targets_np = b_labels.to('cpu').numpy()

            # Append the labels to a numpy list
            targets_list.extend(targets_np)

            if j == 0:  # first batch
                stacked_val_preds = val_preds

            else:
                stacked_val_preds = np.vstack((stacked_val_preds, val_preds))
                
                
                
        # .........................................
        # Calculate the val accuracy for this fold
        # .........................................      


        # Calculate the validation accuracy
        y_true = targets_list
        y_pred = np.argmax(stacked_val_preds, axis=1)

        val_acc = accuracy_score(y_true, y_pred)
        
        
        epoch_acc_scores_list.append(val_acc)


        print('Val loss:' ,total_val_loss)
        print('Val acc: ', val_acc)
        
        
        # .........................
        # Save the best model
        # .........................
        
        if epoch == 0:
            
            # Save the Model
            model_name = 'model_' + str(fold_index) + '.bin'
            torch.save(model.state_dict(), model_name)
            print('Saved model as ', model_name)
            
        if epoch != 0:
        
            val_acc_list = fold_val_acc_list[fold_index]
            best_val_acc = max(val_acc_list)
            
            if val_acc > best_val_acc:
                # save the model
                model_name = 'model_' + str(fold_index) + '.bin'
                torch.save(model.state_dict(), model_name)
                print('Val acc improved. Saved model as ', model_name)
                
                
                
        # .....................................
        # Save the val_acc for this fold model
        # .....................................
        
        # Note: Don't do this before the above 'Save Model' code or 
        # the save model code won't work. This is because the best_val_acc will
        # become current val accuracy.
                
        # fold_val_acc_list is a list of lists.
        # Each fold model has it's own list corresponding to the fold index.
        # Here we choose a list corresponding to the fold number and append the acc score to that list.
        fold_val_acc_list[fold_index].append(val_acc)
        
            

        # Use the garbage collector to save memory.
        #gc.collect()
        
        
    # .............................................................
    # Calculate the CV accuracy score over all folds in this epoch
    # .............................................................   
        
        
    # Print the average val accuracy for all 5 folds
    cv_acc = sum(epoch_acc_scores_list)/NUM_FOLDS_TO_TRAIN
    print("\nCV Acc:", cv_acc)


Num folds used for training: 3
======== Epoch 1 / 3 ========

== Fold Model 0


Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Training...


/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Train loss: 1272.5950364808086

Validation...
Val loss: 282.55339979034034
Val acc:  0.8968835930339139
Saved model as  model_0.bin

== Fold Model 1


...

Val loss: 419.7637463451247
Val acc:  0.8817598533455545

== Fold Model 2
Training...


/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input si

Train loss: 886.0440618107386

Validation...


/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input si

Val loss: 324.51705955329817
Val acc:  0.8909257561869844

CV Acc: 0.8864955698136266
CPU times: user 3h 3min 53s, sys: 1h 30min 13s, total: 4h 34min 6s
Wall time: 4h 32min 40s


In [30]:
epoch_acc_scores_list

[0.886801099908341, 0.8817598533455545, 0.8909257561869844]

In [31]:
fold_val_acc_list

[[0.8968835930339139, 0.9078826764436297, 0.886801099908341],
 [0.8936755270394133, 0.8854262144821264, 0.8817598533455545],
 [0.9042163153070577, 0.8945921173235564, 0.8909257561869844],
 [],
 []]